In [4]:
def test1(**kwargs):
    print(kwargs)
    keys, values = zip(*kwargs.items())
    print(keys)
    print(values)
    keys_list = list(keys)
    print(keys_list)

test1(a=1,b=2,c=3)

{'a': 1, 'b': 2, 'c': 3}
('a', 'b', 'c')
(1, 2, 3)
['a', 'b', 'c']


In [24]:
import tushare as ts
import os
from dotenv import load_dotenv
import pandas as pd
def get_stock_basics():
    available_fields_basic = ['ts_code','symbol','name','area','industry','market','list_status','list_date','delist_date']
    available_fields_info = ['ts_code','main_business']
    load_dotenv()
    api_key = os.environ.get("API_KEY")
    ts.set_token(api_key)
    pro = ts.pro_api()
    # print(api_key)
    stock_basics = pd.concat([pro.stock_basic(fields=','.join(available_fields_basic),list_status='L'), #上市
                             pro.stock_basic(fields=','.join(available_fields_basic),list_status='D'),  #退市
                             pro.stock_basic(fields=','.join(available_fields_basic),list_status='P')], #暂停上市
                             axis=0
                             )
    stock_company_info = pro.stock_company(fields=','.join(available_fields_info))
    print(len(stock_basics))
    print(len(stock_company_info))
    basics_code = stock_basics['ts_code'].tolist()
    info_code = stock_company_info['ts_code'].tolist()
    only_basics = [x for x in basics_code if x not in info_code]
    # print(only_basics)
    only_info = [x for x in info_code if x not in basics_code]
    # print(only_info)
    stock_data = pd.merge(stock_basics, stock_company_info, how='left', on='ts_code')
    return stock_data
get_stock_basics()


5732
6044


,ts_code,symbol,name,area,industry,market,list_status,list_date,delist_date,main_business
0,000001.SZ,000001,平安银行,深圳,银行,主板,L,19910403,None,"主要业务:贷款,拆借,存放等同业业务,债券投资."
1,000002.SZ,000002,万科A,深圳,全国地产,主板,L,19910129,None,房地产开发和物业服务
2,000004.SZ,000004,*ST国华,深圳,软件服务,主板,L,19910114,None,"主要业务:移动互联网游戏业务,移动应用安全服务业务"
3,000006.SZ,000006,深振业A,深圳,区域地产,主板,L,19920427,None,"主要产品或提供的服务:提供商品住宅,以及提供物业租赁服务."
4,000007.SZ,000007,全新好,深圳,其他商业,主板,L,19920413,None,主营业务:物业管理和房屋租赁业
...,...,...,...,...,...,...,...,...,...,...
5727,688555.SH,688555,退泽达(退),None,None,科创板,D,20200623,20230707,医药健康信息化
5728,832317.BJ,832317,观典防务(退),None,None,北交所,D,20200727,20220426,NaN
5729,833874.BJ,833874,泰祥股份(退),None,None,北交所,D,20200727,20220718,NaN
5730,833994.BJ,833994,翰博高新(退),None,None,北交所,D,20200727,20220725,NaN


In [58]:
df_uncleaned = get_stock_basics()
df = df_uncleaned.astype(object).where(pd.notnull(df_uncleaned), None)

5732
6044


In [59]:
print(len(df))
cols = [f"`{col}`" for col in df.columns]
print(cols)
cols_str = ", ".join(cols)
print(cols_str)
placeholders = ", ".join(f":{col}" for col in df.columns)
print(placeholders)

5732
['`ts_code`', '`symbol`', '`name`', '`area`', '`industry`', '`market`', '`list_status`', '`list_date`', '`delist_date`', '`main_business`']
`ts_code`, `symbol`, `name`, `area`, `industry`, `market`, `list_status`, `list_date`, `delist_date`, `main_business`
:ts_code, :symbol, :name, :area, :industry, :market, :list_status, :list_date, :delist_date, :main_business


In [60]:
primary_key = 'ts_code' # 根据您的表结构，主键是 ts_code
update_cols = []
for col in df.columns:
    if col != primary_key:
        update_cols.append(f"`{col}`=VALUES(`{col}`)")
update_str = ", ".join(update_cols)
print(update_str)

`symbol`=VALUES(`symbol`), `name`=VALUES(`name`), `area`=VALUES(`area`), `industry`=VALUES(`industry`), `market`=VALUES(`market`), `list_status`=VALUES(`list_status`), `list_date`=VALUES(`list_date`), `delist_date`=VALUES(`delist_date`), `main_business`=VALUES(`main_business`)


In [61]:
sql_query = f"""
    INSERT INTO `stock_basic_info` ({cols_str})
    VALUES ({placeholders})
    ON DUPLICATE KEY UPDATE {update_str}
"""
print(sql_query)


    INSERT INTO `stock_basic_info` (`ts_code`, `symbol`, `name`, `area`, `industry`, `market`, `list_status`, `list_date`, `delist_date`, `main_business`)
    VALUES (:ts_code, :symbol, :name, :area, :industry, :market, :list_status, :list_date, :delist_date, :main_business)
    ON DUPLICATE KEY UPDATE `symbol`=VALUES(`symbol`), `name`=VALUES(`name`), `area`=VALUES(`area`), `industry`=VALUES(`industry`), `market`=VALUES(`market`), `list_status`=VALUES(`list_status`), `list_date`=VALUES(`list_date`), `delist_date`=VALUES(`delist_date`), `main_business`=VALUES(`main_business`)



In [77]:
data_list_of_dicts = df.to_dict(orient='records')
print(data_list_of_dicts[:3])

[{'ts_code': '000001.SZ', 'symbol': '000001', 'name': '平安银行', 'area': '深圳', 'industry': '银行', 'market': '主板', 'list_status': 'L', 'list_date': '19910403', 'delist_date': None, 'main_business': '主要业务:贷款,拆借,存放等同业业务,债券投资.'}, {'ts_code': '000002.SZ', 'symbol': '000002', 'name': '万科A', 'area': '深圳', 'industry': '全国地产', 'market': '主板', 'list_status': 'L', 'list_date': '19910129', 'delist_date': None, 'main_business': '房地产开发和物业服务'}, {'ts_code': '000004.SZ', 'symbol': '000004', 'name': '*ST国华', 'area': '深圳', 'industry': '软件服务', 'market': '主板', 'list_status': 'L', 'list_date': '19910114', 'delist_date': None, 'main_business': '主要业务:移动互联网游戏业务,移动应用安全服务业务'}]


In [78]:
import sqlalchemy
from src.utils.utils import *
engine = easyConnect()
try:
    with engine.connect() as conn:
        with conn.begin() as transaction:
            conn.execute(sqlalchemy.text(sql_query), data_list_of_dicts)
            transaction.commit()
        print(f"成功向表格 'stock_basic_info' 同步了 {len(df)} 条数据。")
except Exception as e:
    print(f"写入数据库时发生错误:{e}")

成功向表格 'stock_basic_info' 同步了 5732 条数据。
